In [1]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir = '/mnt/share/costeffectiveness/results/vivarium_ciff_sam/v7.1.0_full_model_vicious_cyle_diarrhea_remission/ciff_sam/2022_03_08_00_15_24/count_data/'

In [3]:
six_to_59_ages = ['6-11_months',
       '12_to_23_months', '2_to_4']
ages = ['6-11_months',
       '12_to_23_months', '2_to_4']

In [4]:
os.listdir(output_dir)

['wasting_transition_count.hdf',
 'diarrhea_transition_count.csv',
 'wasting_transition_count.csv',
 'diarrhea_state_person_time.hdf',
 'wasting_state_person_time.hdf',
 'deaths.csv',
 'diarrhea_state_person_time.csv',
 'wasting_state_person_time.csv',
 'deaths.hdf',
 'disease_transition_count.csv',
 'births.csv',
 'population.csv',
 'stunting_state_person_time.hdf',
 'ylds.csv',
 'ylls.csv',
 'disease_state_person_time.csv',
 'population.hdf',
 'diarrhea_transition_count.hdf',
 'ylls.hdf',
 'ylds.hdf',
 'disease_state_person_time.hdf',
 'births.hdf',
 'disease_transition_count.hdf',
 'stunting_state_person_time.csv']

In [5]:
transitions = pd.read_csv(output_dir + 'wasting_transition_count.csv').drop(columns='Unnamed: 0')
pt = pd.read_csv(output_dir + 'wasting_state_person_time.csv').drop(columns='Unnamed: 0')

In [6]:
transitions.head()

,sex,year,measure,input_draw,scenario,value,diarrhea,sq_lns,mam_treatment,sam_treatment,age
0,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,covered,covered,early_neonatal
1,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat1,covered,covered,covered,early_neonatal
2,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,uncovered,covered,early_neonatal
3,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat1,covered,uncovered,covered,early_neonatal
4,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,covered,uncovered,early_neonatal


In [7]:
transitions.measure.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
       'mild_child_wasting_to_susceptible_to_child_wasting_event_count',
       'moderate_acute_malnutrition_to_mild_child_wasting_event_count',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count',
       'severe_acute_malnutrition_to_mild_child_wasting_event_count',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count',
       'susceptible_to_child_wasting_to_mild_child_wasting_event_count'],
      dtype=object)

In [16]:
mam_recovery_transitions = (transitions.loc[(transitions.measure=='moderate_acute_malnutrition_to_mild_child_wasting_event_count')
                                           &(transitions.age.isin(ages))]
                            .groupby(['input_draw','scenario','mam_treatment']).sum())
mam_pt = (pt.loc[(pt.cause=='moderate_acute_malnutrition')&(pt.age.isin(ages))]
          .groupby(['input_draw','scenario','mam_treatment']).sum())

mam_remission = (365 / (mam_recovery_transitions / mam_pt)[['value']]).reset_index()
mam_remission_covered = mam_remission.loc[mam_remission.mam_treatment=='covered'].set_index(['input_draw','scenario'])[['value']]
mam_remission_uncovered = mam_remission.loc[mam_remission.mam_treatment=='uncovered'].set_index(['input_draw','scenario'])[['value']]
mam_difference = mam_remission_covered - mam_remission_uncovered
mam_difference = mam_difference.groupby(['scenario']).describe(percentiles=[0.025,0.975])
mam_difference

value                                             \
                     count       mean       std        min       2.5%   
scenario                                                                
baseline              11.0 -35.157448  3.140781 -39.424392 -39.121708   
lbwsg_interventions   11.0 -36.776461  1.058383 -38.128641 -38.071165   
mam_treatment         11.0 -36.791032  1.075741 -38.048562 -38.037391   
sam_treatment         11.0 -35.155276  3.205407 -39.604246 -39.246967   
sqlns                 11.0 -36.778693  1.058699 -38.155357 -38.093206   
zinc_supplementation  11.0 -36.756344  1.105899 -38.229159 -38.127434   

                                                       
                            50%      97.5%        max  
scenario                                               
baseline             -36.462776 -29.750423 -29.469531  
lbwsg_interventions  -36.626285 -35.171055 -35.055682  
mam_treatment        -36.543156 -35.259396 -35.118724  
sam_treatment        -36.439491 -29.579341 -29.264717  
sqlns                -36.642864 -35.176046 -35.077583  
zinc_supplementation -36.625638 -35.101793 -34.934780

In [ ]:
# MAM

# baseline -35.2 (-29.8 to -39.1) days
# intervention -36.8 (-35.5 to 38.1) days

In [19]:
sam_recovery_transitions = (transitions.loc[(transitions.measure.isin(['severe_acute_malnutrition_to_mild_child_wasting_event_count',
                                                                      'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count']))
                                           &(transitions.age.isin(ages))]
                            .groupby(['input_draw','scenario','sam_treatment']).sum())
sam_pt = (pt.loc[(pt.cause=='severe_acute_malnutrition')&(pt.age.isin(ages))]
          .groupby(['input_draw','scenario','sam_treatment']).sum())

sam_remission = (365 / (sam_recovery_transitions / sam_pt)[['value']]).reset_index()
sam_remission_covered = sam_remission.loc[sam_remission.sam_treatment=='covered'].set_index(['input_draw','scenario'])[['value']]
sam_remission_uncovered = sam_remission.loc[sam_remission.sam_treatment=='uncovered'].set_index(['input_draw','scenario'])[['value']]
sam_difference = sam_remission_covered - sam_remission_uncovered
sam_difference = sam_difference.groupby(['scenario']).describe(percentiles=[0.025,0.975])
sam_difference

value                                                \
                     count       mean        std         min        2.5%   
scenario                                                                   
baseline              11.0 -91.339211  39.276879 -186.847096 -173.866941   
lbwsg_interventions   11.0 -92.135438  39.526522 -187.067331 -174.633560   
mam_treatment         11.0 -92.283352  39.801859 -188.352333 -175.472594   
sam_treatment         11.0 -92.168577  39.880681 -188.597535 -175.657606   
sqlns                 11.0 -92.206922  39.566653 -187.193828 -174.748875   
zinc_supplementation  11.0 -92.310284  39.884523 -188.365102 -175.750798   

                                                       
                            50%      97.5%        max  
scenario                                               
baseline             -76.833316 -52.880145 -50.956343  
lbwsg_interventions  -75.653930 -52.325970 -50.548732  
mam_treatment        -75.376484 -52.364478 -50.477125  
sam_treatment        -75.833193 -52.401667 -50.604772  
sqlns                -75.654690 -52.332936 -50.544051  
zinc_supplementation -75.825614 -52.423770 -50.777966

In [ ]:
# baseline -91.3 (-52.9 to -173.7) days
# intervention -92.1 (-52.3 to -174.6) days